In [48]:
import torch
import numpy as np
import imageio
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader

In [49]:
from dataset import get_rays

# Camera/Dataset

In [50]:
batch_size = 1024
o, d, t = get_rays('fox', mode='train')
dataloader = DataLoader(torch.cat((torch.from_numpy(o),
                                    torch.from_numpy(d),
                                     torch.from_numpy(t)), dim=1), batch_size=batch_size, shuffle=True)


test_o, test_d, test_t = get_rays('fox', mode='test')

In [51]:
test_o.shape

(10, 160000, 3)

In [52]:
# so we can say we have 10 test images and 400*400 images and 3 dimension

# Rendering

# Model

# Training